In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
import boto3
from nba_api.stats.endpoints import LeagueGameFinder, LeagueDashPlayerBioStats
import psycopg2


pd.set_option('display.max_columns', 500)

# Example usage
db_name = "thebigone"
username = "ajwin"
password = "CharlesBark!23"
host = "nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com"


In [2]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]


In [3]:
# CREATE DATABASE
# import boto3

# # Create RDS Client
# rds_client = boto3.client('rds')

# def create_rds_instance(db_instance_identifier, db_name, master_username, master_user_password):
#     try:
#         response = rds_client.create_db_instance(
#             DBInstanceIdentifier=db_instance_identifier,
#             AllocatedStorage=20,  # Adjust as needed
#             DBName=db_name,
#             Engine='postgres',  # Using PostgreSQL, can switch if needed
#             MasterUsername=master_username,
#             MasterUserPassword=master_user_password,
#             DBInstanceClass='db.t3.micro',  # Adjust based on size
#             PubliclyAccessible=False,
#             MultiAZ=False,
#             StorageType='gp2',
#             BackupRetentionPeriod=7,
#         )
#         print(f"RDS instance {db_instance_identifier} creation initiated.")
#         return response
#     except Exception as e:
#         print(f"Error creating RDS instance: {str(e)}")

# create_rds_instance("nba-rds-instance","thebigone","ajwin","CharlesBark!23")


RDS instance nba-rds-instance creation initiated.


{'DBInstance': {'DBInstanceIdentifier': 'nba-rds-instance',
  'DBInstanceClass': 'db.t3.micro',
  'Engine': 'postgres',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'ajwin',
  'DBName': 'thebigone',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '06:40-07:10',
  'BackupRetentionPeriod': 7,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-048e804df880ce6ca',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'default',
   'DBSubnetGroupDescription': 'default',
   'VpcId': 'vpc-0da11b2b953a7cab5',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-02fedebb493b9a06e',
     'SubnetAvailabilityZone': {'Name': 'us-east-1c'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-0c8f0f9dc389265c1',
     'SubnetAvailabilityZone': {'Name': 'us-east-1e'},
     'S

In [6]:
seasons = []
for i in list(range(2020,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]


def get_games_data():
    # Get game data
    gamesl = []
    for i in seasons:
        gamefinder = LeagueGameFinder(league_id_nullable='00',season_nullable=i).get_data_frames()[0]
        gamesl.append(gamefinder)
        time.sleep(1)
    gamehistory = pd.concat(gamesl,axis=0)
    gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])
    return gamehistory


def get_players_data():
    # Get player data
    playersl = []
    for i in seasons:
        playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
        playerfinder['season']=i
        playersl.append(playerfinder)
        time.sleep(.5)
    allplayers = pd.concat(playersl,axis=0)
    return allplayers


In [4]:

# Connect to RDS database
def connect_to_rds(db_name, username, password, host, port=5432):
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=username,
            password=password,
            host=host,
            port=port
        )
        print("Connected to RDS database")
        return conn
    except Exception as e:
        print(f"Error connecting to RDS: {str(e)}")
        return None

# Create a table in PostgreSQL
import psycopg2

# Data type mapping from pandas to PostgreSQL
def map_dtype_to_postgresql(dtype):
    if dtype == 'int64':
        return 'INTEGER'
    elif dtype == 'float64':
        return 'FLOAT'
    elif dtype == 'bool':
        return 'BOOLEAN'
    elif dtype == 'datetime64[ns]':
        return 'TIMESTAMP'
    else:
        return 'TEXT'  # Default to TEXT for object or string types

# Create a table in PostgreSQL based on the dataframe schema
def create_table(conn, table_name, dataframe):
    try:
        cursor = conn.cursor()

        # Generate the SQL statement for creating a table
        columns = []
        for col, dtype in zip(dataframe.columns, dataframe.dtypes):
            pg_type = map_dtype_to_postgresql(dtype.name)
            columns.append(f"{col} {pg_type}")
        
        # Combine all columns into a SQL CREATE TABLE statement
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)});"
        
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table {table_name} created successfully with inferred schema.")
    except Exception as e:
        print(f"Error creating table: {str(e)}")


# Insert data into the table
def insert_data(conn, table_name, dataframe):
    cursor = conn.cursor()
    for _, row in dataframe.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(dataframe.columns)
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"
        cursor.execute(insert_query, tuple(row))
    conn.commit()
    print(f"Data inserted into {table_name}.")

def insert_dataframe(conn, table_name, dataframe):
    try:
        cursor = conn.cursor()

        # Generate the columns and placeholders
        columns = ', '.join(dataframe.columns)
        placeholders = ', '.join(['%s'] * len(dataframe.columns))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING"

        # Convert the dataframe into a list of tuples
        data = [tuple(row) for row in dataframe.itertuples(index=False, name=None)]

        # Use executemany for batch insert
        cursor.executemany(insert_query, data)

        # Commit the transaction
        conn.commit()
        print(f"{len(data)} records inserted into {table_name} successfully.")
    except Exception as e:
        print(f"Error inserting data into {table_name}: {str(e)}")


conn = connect_to_rds(db_name, username, password, host)


Connected to RDS database


In [7]:

# Get and insert games data
games_data = get_games_data()
create_table(conn, "games", games_data)
insert_dataframe(conn, "games", games_data)

# Get and insert players data
players_data = get_players_data()
create_table(conn, "players", players_data)
insert_dataframe(conn, "players", players_data)


Table games created successfully with inferred schema.
10815 records inserted into games successfully.
Table players created successfully with inferred schema.
2256 records inserted into players successfully.


In [5]:

# Function to delete a table from the PostgreSQL database
def delete_table(conn, table_name):
    try:
        cursor = conn.cursor()
        delete_table_query = f"DROP TABLE IF EXISTS {table_name};"
        
        # Execute the query to delete the table
        cursor.execute(delete_table_query)
        conn.commit()
        print(f"Table {table_name} deleted successfully.")
    except Exception as e:
        print(f"Error deleting table {table_name}: {str(e)}")

delete_table(conn,table_name="players")
delete_table(conn,table_name="games")


Table players deleted successfully.
Table games deleted successfully.


In [12]:
#############################################################
#############################################################

# Query the database and return results as a pandas dataframe
def query_database_to_dataframe(conn, query):
    try:
        # Read the query result into a pandas dataframe
        dataframe = pd.read_sql(query, conn)
        return dataframe
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Connect to the RDS database
conn = connect_to_rds(db_name, username, password, host)

# Define your SQL query
query = "SELECT * FROM games WHERE GAME_DATE > '2022-01-01';"  # Example query: fetch 10 rows from the 'games' table

# Execute the query and store the result in a pandas dataframe
if conn:
    result_df = query_database_to_dataframe(conn, query)


# Close the connection
conn.close()

Connected to RDS database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_14404\1965162495.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [13]:
result_df

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,plus_minus
0,42023,1610612742,DAL,Dallas Mavericks,0042300405,2024-06-17,DAL @ BOS,L,240,88,35,78,0.449,11,37,0.297,7,13,0.538,7,28,35,18,4,4,13,20,-18.0
1,42023,1610612738,BOS,Boston Celtics,0042300405,2024-06-17,BOS vs. DAL,W,240,106,38,89,0.427,13,39,0.333,17,20,0.850,15,36,51,25,9,2,7,15,18.0
2,42023,1610612742,DAL,Dallas Mavericks,0042300404,2024-06-14,DAL vs. BOS,W,240,122,46,91,0.505,15,37,0.405,15,22,0.682,13,39,52,21,7,2,8,17,38.0
3,42023,1610612738,BOS,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,L,239,84,29,80,0.363,14,41,0.341,12,13,0.923,4,27,31,18,2,5,13,19,-38.0
4,42023,1610612738,BOS,Boston Celtics,0042300403,2024-06-12,BOS @ DAL,W,240,106,38,82,0.463,17,46,0.370,13,14,0.929,6,30,36,26,4,6,9,19,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7170,22021,1610612756,PHX,Phoenix Suns,0022100549,2022-01-02,PHX @ CHA,W,241,133,52,103,0.505,17,41,0.415,12,12,1.000,14,39,53,36,15,4,17,21,34.0
7171,22021,1610612761,TOR,Toronto Raptors,0022100545,2022-01-02,TOR vs. NYK,W,241,120,41,86,0.477,13,34,0.382,25,30,0.833,11,33,44,21,9,5,16,27,15.0
7172,22021,1610612750,MIN,Minnesota Timberwolves,0022100551,2022-01-02,MIN @ LAL,L,241,103,41,92,0.446,9,38,0.237,12,16,0.750,20,36,56,21,9,2,23,22,-5.0
7173,22021,1610612747,LAL,Los Angeles Lakers,0022100551,2022-01-02,LAL vs. MIN,W,240,108,36,75,0.480,13,41,0.317,23,29,0.793,4,24,28,20,13,5,15,16,5.0
